## Now I got the data for the results with Tensorflow dence layers. I categorized good MAPE result country and not good one. 

Now I will train category 2 with linear regression to get better MAPE results. 

### Things to do:

* Load result data with category. 
* Load train data. 
* Grab the cfips on category 2.
* train with linear regression if the country incategory 2, then update MAPE. 


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from helper_functions import *

In [47]:
data = pd.read_csv("result_dense_with_category.csv")
train = pd.read_csv("train.csv")

In [48]:
# Grab the cfips on category 2
cfips_category1 = data[data['category'] == 1]['Country'].unique()

In [49]:
len(cfips_category1)

609

### Work on this for loop to get new MAPE with new training. Let's see if the total MAPE increase. 

Code above is the Renear regression training. 

In [74]:
data['forecast'].iloc[0][11:-28]

'[3.3091352 3.3172703 3.3280153 3.3369312 3.3491817 3.3590963 3.3678777\n 3.3871744]'

In [26]:
cfips_category1[:5]

array([1005, 1009, 1061, 1069, 1077], dtype=int64)

In [87]:
THRESHOLD = 0.103

IDS = train.cfips.unique()
x_train = np.arange(39).reshape((-1,1))
x_test = np.arange(38,47).reshape((-1,1))

# Based on the Chris's rule, SMAPE > THRESHOLD -> Use linear regrssion 
lr_or_non_lr = []  # If use linear regression, label as 1, if not label as 0
smape = []
mape = []
new_predict = []

for cfips in IDS:
    c = cfips


    # If the cfips in the category 2, then train with linear regression and update MAPE
    if c in cfips_category1:
        
        mape.append(np.float64(data[data['Country'] == c]["MAPE"]))
        new_predict.append(data[data['Country'] == c]['forecast'])
    # If category 1, keep the MAPE score
    else:
        df = train.loc[train['cfips'] == c]
        last = df['microbusiness_density'].values[-1]

        # Fit linear regression
        model = LinearRegression()
        model.fit(x_train,df['microbusiness_density'])
        p = model.predict(x_train)

        # Compute train error
        err = p - df['microbusiness_density'].values
        rng = df['microbusiness_density'].max() - df['microbusiness_density'].min()
        #print(f'Error: {err}\nRange: {rng}')

        # Determin if time series is linear or not
        s = 0
        for k in range(39):
            e = np.abs(err[k])
            #print(f'e : {e}')
            r = e / rng # absolute error divided by range
            #print(f'r: {r}')
            s += r
        #mape.append(s) # Grab MAPE to compare with other data
        #print(f's: {s}')
        s = s/39 # now S is MAPE mean absolute percentage error
        smape.append(s) 
        #print(f'Divide s by the number of data -> MAPE: {s}')

        # Infer test data with linear regression
        p2 = model.predict(x_test)
        shift = last - p2[0]
        if s <THRESHOLD: 
            preds = p2[1:]+shift
            label = 0
            lr_or_non_lr.append(label)
            test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
            results = evaluate_preds(test_label, preds)
            m = results['mape']
            predict.append(preds)
            mape.append(m)
            new_predict.append(preds)
        else: 
            preds = [last]*8
            label = 1
            lr_or_non_lr.append(label)
            test_label = np.expand_dims(np.array(train[train['cfips'] == c]['microbusiness_density'][31:39]), axis=0)
            results = evaluate_preds(test_label, preds)
            m = results['mape']
            predict.append(preds)
            mape.append(m)
            new_predict.append(preds)

count_lr = lr_or_non_lr.count(0)
count_non_lr = lr_or_non_lr.count(1)
print(f'The data predict with Linear Regression: {count_lr}')
print(f'The data not predict with Linear Regression: {count_non_lr}')
print(f'Model: {model}\nMean SMAPE: {np.mean(smape)}\nMin SMAPE: {np.min(smape)}\nMax SMAPE: {np.max(smape)}')
                
        
data['new_MAPE'] = mape
data['new_predict'] = new_predict
data[['MAPE', 'new_MAPE']]

The data predict with Linear Regression: 348
The data not predict with Linear Regression: 2178
Model: LinearRegression()
Mean SMAPE: 0.1621980590423406
Min SMAPE: 0.025774969877374484
Max SMAPE: 0.34610839978243374


,MAPE,new_MAPE
0,1.504614,3.881166
1,4.143341,2.712604
2,1.395184,1.395184
3,5.228748,1.755025
4,1.442244,1.442244
...,...,...
3130,8.937141,0.594611
3131,3.437833,1.642972
3132,4.339747,1.179851
3133,2.373714,2.907186


#### Check how many MAPE incresed

In [88]:
print(f"Old Mean MAPE: {data['MAPE'].mean()}\nNew Mean MAPE: {data['new_MAPE'].mean()}")

Old Mean MAPE: 27.99670683904312
New Mean MAPE: 3.229799900521621


In [95]:
increased = 0
unchanged = 0
decreased = 0
for i in range(len(data)):
    if data['MAPE'].iloc[i] > data['new_MAPE'].iloc[i]:
        increased += 1
    elif data['MAPE'].iloc[i] < data['new_MAPE'].iloc[i]:
        decreased += 1
    else:
        unchanged += 1
print(f"MAPE Increased: {increased}\nMAPE Decreased: {decreased}\nUnchanged: {unchanged}")

MAPE Increased: 1959
MAPE Decreased: 567
Unchanged: 609


## Conclusion:

MAPE Increased significantly by using Linear Regression with the SMAPE < 1.03, and use the last microbusiness density value for the SMAPE larger than 1.03 with the Country I categorised 2 after I trained on tensorflow Dense layer result. See if this model will increase submission. 

* Train full data on Tensorflow Dense model for the category 1. 
* Train full data on Linea Regression on category 2. 


In [89]:
data.to_csv("result_after_LR.csv", index=False)

In [90]:
d = pd.read_csv("result_after_LR.csv")

In [91]:
d.head()

,Country,MAPE,Density,Active,forecast,category,new_MAPE,new_predict
0,1001,1.504614,3.463856,1472,tf.Tensor(\n[3.3091352 3.3172703 3.3280153 3.3...,2,3.881166,[3.47705964 3.49026379 3.50346793 3.51667207 3...
1,1003,4.143341,8.359798,14320,tf.Tensor(\n[7.858338 7.8851886 7.9074492 7.9...,2,2.712604,"[8.3597984, 8.3597984, 8.3597984, 8.3597984, 8..."
2,1005,1.395184,1.232074,244,tf.Tensor(\n[1.1614107 1.1651304 1.1701925 1.1...,1,1.395184,2 tf.Tensor(\n[1.1614107 1.1651304 1.170192...
3,1007,5.228748,1.287240,229,tf.Tensor(\n[1.222997 1.2201089 1.2209381 1.2...,2,1.755025,"[1.28724, 1.28724, 1.28724, 1.28724, 1.28724, ..."
4,1009,1.442244,1.831783,813,tf.Tensor(\n[1.7523701 1.7636061 1.7706515 1.7...,1,1.442244,4 tf.Tensor(\n[1.7523701 1.7636061 1.770651...
